<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Cristóbal Alcázar
- Nombre de alumno 2: Gianina Salomó


### **Link de repositorio de GitHub:** `https://github.com/alcazar90/sci-prog-lab`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [27]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import r2_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 25.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.10.0
    Uninstalling plotly-5.10.0:
      Successfully uninstalled plotly-5.10.0


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [28]:
df_players = pd.read_csv('https://raw.githubusercontent.com/alcazar90/sci-prog-lab/main/data/stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

- Primero se observa la frecuencia de datos perdidos y del resto de las categorías.
- Con ello, es posible concluir que el vector objetivo a construir tiene clases muy desbalanceadas, luego se corrobora explorando las frecuencias.

In [29]:
# Se explora frecuencias de la variable solicitada
df_players["National_Position"].value_counts("%", dropna=False) * 100

NaN    93.887878
Sub     3.161246
LCB     0.272913
GK      0.267228
RCB     0.261542
LB      0.221742
RB      0.216056
RM      0.193314
LM      0.181942
ST      0.170571
LCM     0.142142
RCM     0.142142
LDM     0.108028
CAM     0.108028
RDM     0.102343
RS      0.102343
LS      0.102343
CB      0.051171
CDM     0.051171
CM      0.051171
RW      0.039800
LW      0.039800
LWB     0.022743
RAM     0.022743
LAM     0.022743
RWB     0.022743
RF      0.017057
LF      0.017057
Name: National_Position, dtype: float64

In [30]:
df_players["target"] = pd.Series(np.where(df_players["National_Position"].isna(),
                                "no_seleccionado", "seleccionado")).astype('category')
print(f'Frecuencia relativa vector objetivo:\n{df_players["target"].value_counts("%").round(2) * 100}\n')
fig = df_players["target"].plot(kind="hist", title="Frecuencia absoluta vector objetivo");
fig.layout.bargap = 0
fig.show();

Frecuencia relativa vector objetivo:
no_seleccionado    94.0
seleccionado        6.0
Name: target, dtype: float64



### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [x] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [x] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [x] Crear 3 ``pipelines`` con diferentes clasificadores.
- [x] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [x] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta**

In [31]:
# Se define listas según tipo de columna considerando el análisis entregado y
# la información disponible en el sitio kaggle.
num_cols = [
    "Height", "Weight", "Age", "Ball_Control", "Dribbling", "Marking",
    "Sliding_Tackle", "Standing_Tackle", "Aggression", "Reactions",
    "Interceptions", "Vision", "Composure", "Crossing", "Short_Pass",
    "Long_Pass", "Acceleration", "Speed", "Stamina", "Strength", "Balance",
    "Agility", "Jumping", "Heading", "Shot_Power", "Finishing", "Long_Shots",
    "Curve", "Freekick_Accuracy", "Penalties", "Volleys"
]

cat_cols = [
    "Nationality", "Club_Position", "Preffered_Foot", "Work_Rate",
    "Weak_foot", "Skill_Moves", 
]

In [32]:
# Genere un ColumnTransformer enfocado en preprocesar los datos

# Pipeline con transformaciones para columnas númericas
num_pipe = Pipeline(
    steps=[
        ("imputer_num", KNNImputer(n_neighbors=2, weights="uniform")),
        ("scaler", StandardScaler()),
    ]
)

# Pipeline con transformaciones para columnas categóricas
cat_pipe = Pipeline(
    steps=[
        ("imputer_cat", SimpleImputer(strategy="constant", fill_value="missing")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

# Las transformaciones se aplican en tuplas que asocian Pipelines a grupos de
# columnas
preprocessing = Pipeline([
    ("column_transform", ColumnTransformer(transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ]
  ))
])

**Indicar y Justificar que preprocesamiento utiliza sobre cada columna.**

- Se agrega variables para _pipelines_ específicos de procesamiento según tipo de dato (`num_pipe` y `cat_pipe`). Esto con el fin de especificar para cada caso el procesamiento e imputación adecuados:
    - ***Numéricas***: Se considera las columnas con valores continuos entregadas en el análisis exploratorio. Debido a que no todas presentan la misma unidad de medición, se aplica el proceso de estandarización mediante puntaje Z con `StandardScaler`. A su vez, se agrega el paso de imputar valores nulos, en caso de que estos se presenten en futuros datos para predicciones.

    - ***Categóricas***: Se considera las columnas categóricas mencionadas en el análisis exploratorio, _omitiendo el nombre_ del jugador, ya que este no se considera relevante para el análisis. También se elimina _National_Position_, debido a su colinealidad con el vector objetivo. Para el resto, se asigna además un paso de imputación de nulos, para aplicarse en caso de que estos se presenten para predicciones de datos futuros. Además, se aplica una codificación de tipo `OneHotEncoder` para generar columnas binarias para cada categoría, ya que los modelos escogidos no presentan problemas frente a multicolinealidad de atributos.

In [33]:
# Crear 3 pipelines con diferentes clasificadores
knn_pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing), 
        ("knn", KNeighborsClassifier(n_neighbors=3))
    ]
)

tree_pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing), 
        ("rf", RandomForestClassifier(random_state=42, ccp_alpha=0.00015))
    ]
)

svm_pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing), 
        ("svm", SVC(kernel="poly"))
    ]
)

**Justificación de la selección de clasificadores**

Dado que se trata de un problema supervisado de clasificación binaria, con menos de 100.000 muestras, se escoge los siguientes clasificadores:

- _KNN_: Se escoge por su simpleza y obtener en general resultados aceptables. Se decide utilizar 3 vecinos.
- _RFC_: Se escoge por su interpretabilidad, aunque debido a su tendencia a generar overfitting se aplica una regularización mediante `ccp_alpha=0.00015`
- _SVC_: Se escoge con uso de kernel polinomial de grado 3 y valores de regularización por defecto. Esto con el objetivo de ver si al aplicar una transformación en los datos se obtiene métricas predictivas diferentes respecto de los modelos anteriores.

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar los `pipelines`.
- [X] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [34]:
# Separar el conjunto de datos en entrenamiento y de prueba
X = df_players.drop(columns=["Name", "National_Position", "target"])
y = df_players["target"].replace(["seleccionado", "no_seleccionado"], [1, 0])

# Se realiza un muestreo estratificado para conversar distribución de las clases
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
                                                    random_state=42,
                                                    test_size=0.2)

In [35]:
X_train.shape

(14070, 37)

In [36]:
#@title
print('Se conserva distribución de etiquetas:\n---------------------------------------------')
print('Proporción Train:\n', y_train.value_counts('%')*100, '\n')
print('Proporción Test:\n', y_train.value_counts('%')*100, '\n')
print('Proporción Original:\n', y.value_counts('%')*100)

Se conserva distribución de etiquetas:
---------------------------------------------
Proporción Train:
 0    93.887704
1     6.112296
Name: target, dtype: float64 

Proporción Test:
 0    93.887704
1     6.112296
Name: target, dtype: float64 

Proporción Original:
 0    93.887878
1     6.112122
Name: target, dtype: float64


In [37]:
%%capture
# Entrenar los pipelines
knn_pipe.fit(X_train, y_train)
tree_pipe.fit(X_train, y_train)
svm_pipe.fit(X_train, y_train)

In [38]:
# Utilizar las métricas para evaluar los modelos generados

for p in [knn_pipe, tree_pipe, svm_pipe]:
  print(f"Métricas para modelo {list(p.named_steps.keys())[1]} en Test:\n")
  print(classification_report(y_test, p.predict(X_test)))

Métricas para modelo knn en Test:

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      3303
           1       0.48      0.13      0.20       215

    accuracy                           0.94      3518
   macro avg       0.71      0.56      0.58      3518
weighted avg       0.92      0.94      0.92      3518

Métricas para modelo rf en Test:

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       0.71      0.10      0.18       215

    accuracy                           0.94      3518
   macro avg       0.83      0.55      0.57      3518
weighted avg       0.93      0.94      0.92      3518

Métricas para modelo svm en Test:

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       0.76      0.09      0.16       215

    accuracy                           0.94      3518
   macro avg       0.85 

In [39]:
for p in [knn_pipe, tree_pipe, svm_pipe]:
  print(f"Métricas para modelo {list(p.named_steps.keys())[1]} en Train:\n")
  print(classification_report(y_train, p.predict(X_train)))

Métricas para modelo knn en Train:

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     13210
           1       0.85      0.30      0.44       860

    accuracy                           0.95     14070
   macro avg       0.90      0.65      0.71     14070
weighted avg       0.95      0.95      0.94     14070

Métricas para modelo rf en Train:

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     13210
           1       1.00      0.34      0.51       860

    accuracy                           0.96     14070
   macro avg       0.98      0.67      0.75     14070
weighted avg       0.96      0.96      0.95     14070

Métricas para modelo svm en Train:

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     13210
           1       0.99      0.17      0.30       860

    accuracy                           0.95     14070
   macro avg       0.

**Respuesta**:

**¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?**

- _Accuracy_: Corresponde a la cantidad de predicciones correctas por sobre la cantidad total de predicciones. Es decir, es una métrica de desempeño global del modelo, no exclusivamente de la clase positiva. Por tanto, dado que en este caso la clase positiva tiene una frecuencia mucho menor, _el valor obtenido de accuracy no es representativo para la clase positiva_. En los tres modelos se obtiene valores similares y altos tanto en train como test, pero dado el desbalance de clases, no se puede considerar en base a este valor que el modelo sea adecuado para predecir si un jugador ese seleccionado o no.
- _Precisión_: Fracción de ejemplos asignados a la clase `i` que son realmente de la clase `i`. En el caso del reporte de clasificación, se entrega los valores obtenidos considerando cada clase como positiva en forma independiente para obtener su precission en cada una. En este caso, nos fijamos por tanto en el valor reportado para la clase `1`. Bajo este criterio, se obtiene el mejor desempeño para el modelo `SVC`. Se debe considerar igual que el valor obtenido en Test es mucho menor que en Train, indicando un sobreajuste del modelo que debiese manejarse con regularización.
- _Recall_: Fracción de ejemplos de la clase `i` correctamente clasificados. Nuevamente nos fijamos en el valor reportado para la clase `1`. Bajo este criterio, los tres clasificadores entregan un valor bajo, tanto en Train como Test, lo que indicaría presencia de falsos negativos en las predicciones.


**¿Podrían mejorarse los resultados?, ¿Cómo?**

Sí. En al desbalance de clases, la primera alternativa siempre es conseguir más datos (reales). Asumiendo que estos no son posibles de obtener, se puede efectuar un balanceo artifical de clases para los datos de entrenamiento del modelo. Esto puede ser por ejemplo aplicando `RandomUnderSampler`, para dismiuir los datos de la clase `no_seleccionado`, y de esa forma balancear la proporción con los clase `seleccionado` en los datos de entrenamiento. 

En cuanto a los atributos, se podría también explorar hacer una ingeniería de atributos que permita disminuir la dimensionalidad, con la esperanza de mejorar el desempeño del clasificador. Puede ser en forma manual agrupando categorías (Ej, pasar nacionalidad a continentes), o mediante métricas de scoring como anova o mutual information.

En cuanto a los modelos, es posible también explorar otras combinaciones de hiperparámetros. Por último, siempre es bueno considerar la regularización y comparar el desempeño del modelo tanto en Train como Test para corroborar que no se está frente a un modelo sobreajustado, y que por ende será generalizable frente a predicciones futuras.


**¿Influye la cantidad de ejemplos por clase?**

Sí, en todos los casos los modelos sufren una degradación en su desempeño
frente al escenario de clases desbalanceadas. En general, los modelos se
benefician al obtener mayor información por categoría, y en el peor de los
casos, la cantidad de información solo puede ser redundante. Siempre se debe
considerar que clases desbalanceadas genera un sesgo a favor de la clase
mayoritaría en los resultados, como se corroborá con las métricas reportadas de
la clase mayoritaría (0) versus la minoritaría (1). Existen técnicas de pre-procesamiento para aminorar este problema, como la que se mencionó anteriormente.


## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [X] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [X] Comente los resultados obtenidos.

**Respuesta:**

Utilizamos un diccionario para crear las nuevas categorías que
buscamos predecir y creamos la columna `target_multiclase`:



In [40]:
# Aplique las etiquetas descritas anteriormente en cada uno de los valores
# señalados en esta sección.
macro_categories = {
    'ST':'ataque',
    'CF':'ataque',
    'RW':'central_ataque',
    'CAM':'central_ataque',
    'LW':'central_ataque',
    'RM':'central',
    'CM':'central',
    'LM':'central',
    'RWB':'central_defensa',
    'CDM':'central_defensa',
    'LWB':'central_defensa',
    'RB':'defensa',
    'CB':'defensa',
    'LB':'defensa',
    'GK':'arquero'
}

# Se crean macro-categorías según las clases específicadas
df_players['target_multiclase'] = (df_players['Club_Position']
  .astype('category')
  .replace(macro_categories)
)

Observemos la cantidad de jugadores por cada categoría en la nueva columna `target_multiclase`.

In [41]:
# Cuente cuantos por clase quedan
df_players['target_multiclase'].value_counts(dropna=False)

Sub                7492
Res                3146
defensa            1180
central             907
RCB                 633
arquero             632
LCB                 631
central_ataque      581
ataque              430
LCM                 354
RCM                 353
LDM                 266
RDM                 266
central_defensa     209
RS                  206
LS                  205
LAM                  36
RAM                  36
LF                   12
RF                   12
NaN                   1
Name: target_multiclase, dtype: int64

Filtrar solo los jugadores que se encuentren en las categorías relevantes
específicadas en el enunciado, y luego
separar el _dataset_ en conjunto de entrenamiento y prueba. Se vuelve a utilizar
un muestreo estratificado para mantener la distribución de las categorías
en `target_multiclase`.

Debido a la baja cantidad de registros que cumplen con el requisito solicitado, se cambia la fracción considerada para datos de test, con el fin de obtener un poco más de datos y así tener métricas de desempeño más confiables. De todas formas, considerando que solo se cuenta con 3.939 registros que cumplen con lo solicitado y se necesitan 6 categorías, la cantidad de datos para cada clase será baja.

In [42]:
# Crear np.array booleano para filtrar solo los jugadores en categorías relevantes
categorias_relevantes = list(set(macro_categories.values()))
mask = np.isin(df_players.target_multiclase, categorias_relevantes)
print(f'Total de jugadores en categorías relevantes: {np.sum(mask)}')

Total de jugadores en categorías relevantes: 3939


In [43]:
# Se utiliza la mask anterior para filtrar las observaciones relevantes
# y eliminar las columnas que no te utilizaran en el entrenamiento
X_cp = df_players[mask].drop(
    columns=["Name", "Club_Position", "target", "target_multiclase"]
)

# Se obtiene la columna con las categorías multiclase
y_cp = df_players[mask]["target_multiclase"]

# Se realiza un muestreo estratificado para conversar distribución de las clases
X_train_cp, X_test_cp, y_train_cp, y_test_cp = train_test_split(
    X_cp, y_cp, stratify=y_cp, random_state=42, test_size=0.33
)

In [44]:
# Cantidad de casos por clase en datos de test
pd.Series(y_test_cp).value_counts()

defensa            389
central            299
arquero            209
central_ataque     192
ataque             142
central_defensa     69
Name: target_multiclase, dtype: int64

- En el caso de las columnas categóricas, para este problema se incorpora la columna `National_Position` y se elimina la columna `Club_Position`, debido a la colinealidad de esta última con el vector objetivo creado para este ejercicio.

In [45]:
# Se define las columnas categóricas para este problema
cat_cols_cp = [
    "Nationality", "National_Position", "Preffered_Foot", "Work_Rate",
    "Weak_foot", "Skill_Moves", 
]

# Las transformaciones se aplican en tuplas que asocian Pipelines a grupos de
# columnas
preprocessing_cp = Pipeline([
    ("column_transform", ColumnTransformer(transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols_cp),
    ]
  ))
])

**Justificación de los modelos seleccionados**

Dado que estamos frente a un problema supervisado de clasificación multiclase, y tenemos menos de 100.000 datos, se escoge los mismos modelos del punto anterior, con la misma justificación. Estos modelos permiten manejar también el problema de clasificación multiclase, es por ello que se puede mantener los mismos.

In [46]:
# Entrene el nuevo pipeline y ejecute una evaluación de este.
knn_pipe_cp = Pipeline(
    [
        ("preprocesamiento", preprocessing_cp), 
        ("knn", KNeighborsClassifier(n_neighbors=3))
    ]
)

tree_pipe_cp = Pipeline(
    [
        ("preprocesamiento", preprocessing_cp), 
        ("rf", RandomForestClassifier(random_state=42, ccp_alpha=0.001))
    ]
)

svm_pipe_cp = Pipeline(
    [
        ("preprocesamiento", preprocessing_cp), 
        ("svm", SVC())
    ]
)

Ajustamos los modelos y reportamos las métricas en conjunto de prueba y entrenamiento.

In [47]:
%%capture
knn_pipe_cp.fit(X_train_cp, y_train_cp)
tree_pipe_cp.fit(X_train_cp, y_train_cp)
svm_pipe_cp.fit(X_train_cp, y_train_cp)

In [48]:
for p in [knn_pipe_cp, tree_pipe_cp, svm_pipe_cp]:
  print(f"Métricas para modelo {list(p.named_steps.keys())[1]} en Test:\n")
  print(classification_report(y_test_cp, p.predict(X_test_cp)))

Métricas para modelo knn en Test:

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.62      0.80      0.70       142
        central       0.50      0.55      0.52       299
 central_ataque       0.34      0.21      0.26       192
central_defensa       0.18      0.07      0.10        69
        defensa       0.82      0.92      0.86       389

       accuracy                           0.68      1300
      macro avg       0.58      0.59      0.57      1300
   weighted avg       0.65      0.68      0.66      1300

Métricas para modelo rf en Test:

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.76      0.82      0.79       142
        central       0.55      0.64      0.59       299
 central_ataque       0.45      0.21      0.29       192
central_defensa       0.75      0.04      0.08        69
        defensa

In [49]:
for p in [knn_pipe_cp, tree_pipe_cp, svm_pipe_cp]:
  print(f"Métricas para modelo {list(p.named_steps.keys())[1]} en Train:\n")
  print(classification_report(y_train_cp, p.predict(X_train_cp)))

Métricas para modelo knn en Train:

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       423
         ataque       0.78      0.94      0.85       288
        central       0.69      0.80      0.74       608
 central_ataque       0.80      0.55      0.65       389
central_defensa       0.77      0.34      0.47       140
        defensa       0.88      0.93      0.90       791

       accuracy                           0.83      2639
      macro avg       0.82      0.76      0.77      2639
   weighted avg       0.83      0.83      0.82      2639

Métricas para modelo rf en Train:

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       423
         ataque       0.94      0.95      0.94       288
        central       0.91      0.89      0.90       608
 central_ataque       0.97      0.86      0.92       389
central_defensa       1.00      0.39      0.56       140
        defen

**Respuesta**:

**Comentarios de los resultados obtenidos**:

* Los tres modelos obtuvieron un _accuracy_ $\geq 68\%$ en el conjunto de pruebas. 
Todos los modelos obtuvieron peores resultados respecto a sus versiones en el
problema anterior. Esto es esperable, a mayor número de
categorías dentro del _dataset_, más complejo el problema.
* El modelo con mejores resultados en el conjunto de pruebas es `SVM`, según `accuracy` y `f1-score`.
* La clase minoritaría (i.e. `centra_defensa`) en todos los clasificadores
obtuvó el menor resultado, según `recall` y `f1-score` en el conjunto de pruebas.
* La clase mayoritaría (i.e. `defensa`) en todos los clasificadores obtuvó
el mejor resultado, según `f1-score` en conjunto de pruebas. A excepción
de la clase `arquero`, la cual en todos los clasificadores es identificada
perfectamente, reportando el `f1-score` máximo posible en el conjunto de pruebas. 


## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [X] Generar un pipeline para la regresión.
- [X] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [X] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

**Explique en qué consiste la métrica $R^2$**

Esta métrica permite medir la _bondad de ajuste_ de un modelo. En concreto, _mide la variabilidad del vector objetivo_ (variable dependiente) en función del cambio en la combinación lineal de atributos (variables independientes) del modelo.

Sus valores van _generalmente_ entre 0 y 1, donde 0 significa que el modelo no explica en nada la varianza del vector objetivo, y 1 indica que el modelo explica perfectamente la varianza del vector objetivo. Por ende, lo ideal es obtener, en un modelo descriptivo, valores cercanos a 1. Sin embargo, frente a un problema de machine learning, donde se busca un modelo predictivo generalizable, un bajo $R^2$ en los datos de entrenamiento no necesariamente es algo malo, ya que corresponde por ende a un modelo _no sobreajustado_, del cual se espera que tenga en promedio predicciones más acertadas en datos difernetes al entrenamiento.

Se le llama también coeficiente de determinación, y es un puntaje que representa la proporción de varianza de los valores predichos por el modelo con respecto a la varianza de los valores reales.


**Datos**


- Primero se carga los datos, eliminando duplicados, y promediando los sueldos. Para ello se asume que los jugadores de nombre repetido y sueldos distintos corresponden a la misma persona, ya que no se tiene otro tipo de identificador único. Por ende, para estos casos se promedia el sueldo.

- Luego se añaden estos datos al conjunto original con las métricas de los jugadores.

In [51]:
# Datos de sueldos
df_sueldos = pd.read_csv(
    'https://raw.githubusercontent.com/alcazar90/sci-prog-lab/main/data/sueldos.csv'
).drop(columns=["Unnamed: 0"]).drop_duplicates()

# Se promedia nombres repetidos (se asume la misma persona por no tener id)
df_sueldos = df_sueldos.groupby(["Player"]).agg({
    "Weekly Salary": "mean"
}).reset_index()
df_sueldos.head()

,Player,Weekly Salary
0,Aaron Cresswell,65966.0
1,Aaron Lennon,46177.0
2,Aaron Leya Iseka,13006.0
3,Aaron Mooy,65966.0
4,Aaron Ramsey,158320.0


In [52]:
# Merge
if "Weekly Salary" not in df_players.columns:
  n_rows = df_players.shape[0]
  df_players = pd.merge(
      df_players,
      df_sueldos.rename(columns={
          "Player": "Name"
      }),
      how="left"
  )

  assert df_players.shape[0] == n_rows

df_players["Weekly Salary"].describe().round()

count       1863.0
mean       53686.0
std        81986.0
min          620.0
25%        14137.0
50%        29685.0
75%        65029.0
max      1538905.0
Name: Weekly Salary, dtype: float64

**Generar un pipeline para la regresión**

Se considerará como atributos todos los disponibles en el conjunto de `df_players`, a excepción del nombre del jugador. Para ello, se utiliza las versiones codificadas de `National_Position` y `Club_position`, generadas en los ejercicios anteriores.

Luego se genera los Pipelines para los modelos `LassoCV`, `SVR` y `RandomForestRegression`, cuyas justificaciones de selección se encuentran más adelante.

Se utiliza `OneHotEncoder` en las variables categóricas para que los modelos
que codifican una de sus categoría como base, es decir, en el parámetro _offset_ o _bias_, no presenten problemas al ser ajustadas.


In [65]:
# Se define las columnas categóricas para este problema
cat_cols_s = [
    "Preffered_Foot", "Work_Rate", "Nationality",
    "Weak_foot", "Skill_Moves", "target", "target_multiclase"
]

# Pipeline con transformaciones para columnas categóricas
cat_pipe_s = Pipeline(
    steps=[
        ("imputer_cat", SimpleImputer(strategy="constant", fill_value="missing")),
        ("onehot", OneHotEncoder(handle_unknown="ignore",
                                 drop="first", sparse = False)),
    ]
)

# Las transformaciones se aplican en tuplas que asocian Pipelines a grupos de
# columnas
preprocessing_s = Pipeline([
    ("column_transform", ColumnTransformer(transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe_s, cat_cols_s),
    ]
  ))
])

**Justificación de los modelos seleccionados**

Dado que se está frente a un problema supervisado de regresión (no se está prediciendo una categoría, sino una cantidad), y se tiene menos de 100.000 datos, se escoge los siguientes modelos:


- Modelo _Least Absolute Shrinkage and Selection Operator_ (**LASSO**): Se utiliza un modelo lineal regularizado, cuyo componente de regularización
en la función de costo es la norma $||\beta||_1$ de los parámetros del
modelo. Este modelo al ser una regresión lineal con una función de costo
regularizada, al ajustar sus parámetros puede obtener estimadores con menor
varianza, logrando una mejor generalización.

- Modelo Random Forest: Un modelo de arboles, con capacidad de encontrar
regiones de datos con relaciones no lineales, y con un mécanismo de
regularización vía el promedio de muchas instancias de árboles regresores, y 
que además utilizan distintos subconjuntos de variables. Es una buena
alternativa para capturar complejidad sin _overfittear_ el modelo.

- Modelo Support Vector Machines (SVM): Un modelo SVM de regresión, con un kernel polinomial de grado 3, lo que permite obtener un predictor con bastante
expresividad y capacidad de capturar estructuras no lineales. Además, se
utiliza un componente de regularización para que no sobreajuste los parámetros
durante el entrenamiento. Modelo con buena expresividad, y mécanismo de
regularización para controlar el _overffiting_ del modelo.

In [161]:
# Pipeline para la regresión
lasso_pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_s), 
        ("lasso", LassoCV(n_alphas=100))
    ]
)

tree_pipe_s = Pipeline(
    [
        ("preprocesamiento", preprocessing_s), 
        ("rf", RandomForestRegressor(random_state=42))
    ]
)

svm_pipe_s = Pipeline(
    [
        ("preprocesamiento", preprocessing_s), 
        ("svm", SVR(kernel="poly", gamma=3, C=1/15))
    ]
)


**Obtener un regresor con un  R2  superior a  0.35**

Para todos los casos, se omite el atributo `Name` y las columnas `National_Position` y `Club_Position` (estas dos debido a su colinealidad con `target` y `target_multiclase` respectivamente). Luego se elimina todos los registros donde no se cuente con la información de `Weekly Salary`.

Se genera primero la división de muestras para este problema, considerando un 33% de los datos para test, y luego se entrena el pipeline.

Luego se evalúa la métrica solicitada en los conjuntos de Train y Test.

In [162]:
# Selección de atributos
df_sel = df_players.drop(columns=[
    "Name","National_Position", "Club_Position"
]).dropna(subset=["Weekly Salary"]).copy()

df_sel["target_multiclase"] = np.where(
    df_sel["target_multiclase"].isin(categorias_relevantes),
    df_sel["target_multiclase"],
    "otro"
)

df_sel["Weekly Salary"].describe().round()

count       1863.0
mean       53686.0
std        81986.0
min          620.0
25%        14137.0
50%        29685.0
75%        65029.0
max      1538905.0
Name: Weekly Salary, dtype: float64

In [163]:
# División de muestras
X_s_all = df_sel.drop(columns=["Weekly Salary"])
y_s_all = df_sel["Weekly Salary"]

X_train_s_all, X_test_s_all, y_train_s_all, y_test_s_all = train_test_split(
    X_s_all, y_s_all, test_size=0.33, random_state=42
)

In [164]:
# Entrenar pipeline
%%capture
lasso_pipe.fit(X_train_s_all, y_train_s_all)
tree_pipe_s.fit(X_train_s_all, y_train_s_all)
svm_pipe_s.fit(X_train_s_all, y_train_s_all)

In [165]:
for p in [lasso_pipe, tree_pipe_s, svm_pipe_s]:
  print(f"R2 para modelo {list(p.named_steps.keys())[1]} en Test:"
  f" {round(r2_score(y_test_s_all, p.predict(X_test_s_all)), 5)}")

R2 para modelo lasso en Test: 0.2405
R2 para modelo rf en Test: 0.43017
R2 para modelo svm en Test: 0.40527


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:174: UserWarning:

Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:174: UserWarning:

Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_encoders.py:174: UserWarning:

Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros



In [160]:
for p in [lasso_pipe, tree_pipe_s, svm_pipe_s]:
  print(f"R2 para modelo {list(p.named_steps.keys())[1]} en Train:"
  f" {round(r2_score(y_train_s_all, p.predict(X_train_s_all)), 5)}")

R2 para modelo lasso en Train: 0.40739
R2 para modelo rf en Train: 0.95073
R2 para modelo svm en Train: 0.68052


**Respuesta**:

**Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?**

Dos de los modelos seleccionados cumplen los requisitos del señor
Renacin de obtener un $R^2\geq 0.35$ en el conjunto de pruebas, el modelo
RandomForest y SVM registraron $0.43$ y $0.4$ respectivamente. En cambio,
el modelo LASSO solo obtuvó un $R^2=0.24$. Una de las explicaciones a los resultados es que los dos modelos que cumplieron
el requisito, son modelos no lineales, los cuales tienen mayor expresividad
para capturar conjuntos de datos con mayor complejidad. En cambio, el modelo
LASSO, es un modelo líneal en sus parámetros.

Otra explicación se debe a qué tan agresivo fue el mécanismo de regularización de cada modelo, logrando obtener estimadores con menor
varianza (a costa de sesgo). En el caso de Random Forest, su mecánismo 
de regularización es promediar muchas instancias de árboles, y ademas permite
que cada instancia tenga un subconjunto aleatorio de predictores, este 
empleo de la técnica de _bootstrapping_ sobre el cual se apalanca el modelo
RandomForest, permite conseguir estimadores que generalizan mejor, es decir
con menor varianza. En el caso del modelo SVM, se utilizó un kernel polinomial
de grado 3, lo cuál lo equipa con bastante expresividad, y posibilidades
de _overfittiar_ los datos. Sin embargo, se utiliza un componente de costo
en sus parámetros, equivalente a `C=1/15`, y que permite aprovechar
la expresividad del kernel polinomial pero restringiendola a un espacio de soluciones con mayor generalización, versus solo a memorizar los datos. Finalmente, en el caso del modelo LASSO, el cual obtuvó el peor desempeño según $R^2$ reportado en conjunto de pruebas, también tiene una regularización vía la función de costo. Sin embargo, puede que este modelo no logré una regularización tan efectiva, debido a que tampoco hay un número importante de variables predictoras, por lo que la selección implicita de variables no ayuda, y deja al modelo con menor expresividad para capturar la señal.

Existe un espacio de mejora para los modelos, por ejemplo, podemos realizar _feature engineering_
sobre la variable `Nacionalidad` y agrupar todos los jugadores a nivel continental. Esto podría capturar de mejor forma la señal de los sueldos de los
jugadores sobre las distintas ligas a nivel geográfico, donde probablemente
comenzaron su carrera, y que desarrollaron una parte importante de sus
hábilidades técnicas. El problema de utilizar la variable `Nacionalidad`
directamente, es que las variables con alta cardinalidad, como los países, cuentas con muchas categorías que tienen pocas observaciones, por lo que capturar el efecto de esta variable sobre el target puede ser más complejo. 
Esto debido a la introducción de ruído por categorías con poca información. Otra forma de mejorar los resultados, es eliminar los jugadores con sueldos muy altos, quienes pueden degradar el poder de predicción sobre la mayoría de los sueldos que se concentran en menores niveles. De manera alternativa, realizar
una transformanción sobre la variable predictora, o utilizar modelos diferentes
para distintos niveles de sueldo, y luego realizar una regresión con las
observaciones dentro de ese nivel de sueldo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>